In [10]:
import pandas as pd
import numpy as np

import datetime as dt
import string

import matplotlib.pyplot as plt

In [11]:
from xbbg import blp

### Parameters

In [12]:
WRITE_DATA = True

TODAY = dt.datetime.today().strftime('%Y%m%d')  
OUTFILE = f'../build_data/spx_data_{TODAY}.xlsx'

# Get SPX Members

#### BB Call

In [13]:
DT_MEMBERS = TODAY
weights = blp.bds(['SPX Index'],flds=['INDX_MWEIGHT_HIST'],END_DATE_OVERRIDE=DT_MEMBERS)

In [14]:
TICKS_SPX = weights.index_member.str.split().str.get(0)
TICKS_SPX_BB = (TICKS_SPX + [' US Equity']).to_list()
TICKS_SPX = TICKS_SPX.to_list()

weights.set_index('index_member',inplace=True)
weights.index = weights.index.str.split(' ').str[0]

spx_members = pd.DataFrame({'tickers':TICKS_SPX, 'BB tickers':TICKS_SPX_BB})
spx_members.set_index('tickers',inplace=True)

tickerBB_to_ticker = spx_members.reset_index().set_index('BB tickers')['tickers'].to_dict()

## Download Ticker Info

#### BB Call

In [15]:
info_spx = blp.bdp(TICKS_SPX_BB,flds=['SECURITY_NAME','CUR_MKT_CAP'])

In [16]:
info_spx = info_spx.rename(index=tickerBB_to_ticker)
spx_members = spx_members.join(info_spx)

In [17]:
temp = spx_members['cur_mkt_cap']/spx_members['cur_mkt_cap'].sum()

if weights.max().values[0]<.005:
    weights['percent_weight'] = temp

### Save SPX Member Info

In [18]:
OUTFILE_SPX = f'../build_data/spx_members_{TODAY}.xlsx'

if WRITE_DATA:
    with pd.ExcelWriter(OUTFILE_SPX) as writer:  
        spx_members.to_excel(writer, sheet_name= f'spx members {TODAY}')
        weights.to_excel(writer, sheet_name= f'spx weights {DT_MEMBERS}')

# Download Adjusted Prices for all SPX Members

In [19]:
STARTDATE = '2014-12-31'
ENDDATE = '2024-12-31'

FLD = 'PX_LAST'
ADJ = 'all'

#### BB Call. Careful: Running this takes awhile and uses lots of BB Data Limit

In [20]:
prices_raw = blp.bdh(TICKS_SPX_BB,flds=[FLD],start_date=STARTDATE,end_date=ENDDATE,adjust=ADJ)

In [21]:
prices = prices_raw.droplevel(level=1,axis=1)
prices.rename(columns=tickerBB_to_ticker,inplace=True)

# Download Market Benchmarks

In [22]:
FLD_MKT = 'PX_LAST'
FLD_RF = 'PX_LAST'

ADJ = 'all'

TICKS_MKT = ['SPY US Equity','XBTUSD BGN Curncy','USO US Equity','TLT US Equity','IEF US Equity','IYR US Equity','GLD US Equity']
TICKS_RF = ['USGG3M Index','FDTR Index','SOFRRATE Index','SHV US Equity']

mkt = blp.bdh(TICKS_MKT,flds=[FLD_MKT],start_date=STARTDATE,end_date=ENDDATE,adjust=ADJ).droplevel(level=1,axis=1)
rf = blp.bdh(TICKS_RF,flds=[FLD_RF],start_date=STARTDATE,end_date=ENDDATE,adjust=ADJ).droplevel(level=1,axis=1)

### Get Ticker Info

In [23]:
info_mkt = blp.bdp(TICKS_MKT,flds=['SECURITY_NAME'])
info_rf = blp.bdp(TICKS_RF,flds=['SECURITY_NAME'])

### Clean data for export

In [24]:
prices.index.name = 'date'
mkt.index.name = 'date'
rf.index.name = 'date'

info_spx.index.name = 'ticker'
info_mkt.index.name = 'ticker'
info_rf.index.name = 'ticker'

cols = mkt.columns
cols = [col.replace(' US Equity','') for col in cols]
mkt.columns = cols   

## Save Data

Takes a full minute or so to export the SPX member prices

In [25]:
if ADJ=='all':
    spx_sheet_name = f'spx prices adjusted'
else:
    spx_sheet_name = f'spx prices'


if WRITE_DATA:
    with pd.ExcelWriter(OUTFILE) as writer:  
        info_spx.to_excel(writer, sheet_name= 'spx names')
        info_mkt.to_excel(writer, sheet_name= 'market names')
        info_rf.to_excel(writer, sheet_name= 'rate names')

        prices.to_excel(writer, sheet_name=spx_sheet_name)
        mkt.to_excel(writer, sheet_name='benchmarks prices')
        rf.to_excel(writer, sheet_name='rates')


08JAN2025_12:35:24.090 30304:6202961920 WARN apism_apisession.cpp:1326 Apisession.aliveCheck Alive Request Retries exhausted (2) on [localhost:8194 -> 127.0.0.1:53915] 

08JAN2025_12:35:24.092 30304:6204633088 WARN blpapi_platformcontroller.cpp:489 blpapi.session.{1}.platformcontroller Connectivity lost, no connected endpoints. 

08JAN2025_12:35:31.251 30304:6203535360 WARN blpapi_apicmadapter.cpp:341 blpapi.session.{1}.transporttcp.apicmadapter Connection attempt timed out localhost:8194 <127.0.0.1:8194> 

08JAN2025_12:35:31.251 30304:6203535360 WARN blpapi_platformtransporttcp.cpp:138 blpapi.session.{1}.transporttcp<0><localhost:8194> 127.0.0.1, session pool state=Failed 

08JAN2025_12:35:39.170 30304:6203535360 WARN blpapi_apicmadapter.cpp:341 blpapi.session.{1}.transporttcp.apicmadapter Connection attempt timed out localhost:8194 <127.0.0.1:8194> 

08JAN2025_12:35:39.170 30304:6203535360 WARN blpapi_platformtransporttcp.cpp:138 blpapi.session.{1}.transporttcp<0><localhost:8194> 12